In [1]:
%pip install llama-index-llms-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8

In [2]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.6/250.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 22.4 MB/s eta 0:00:00


In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

In [4]:
!mkdir data

In [5]:
# load documents
documents = SimpleDirectoryReader("./data/").load_data()

In [6]:
print(documents)

[Document(id_='8b4c953e-9f48-40da-9136-d33e84073a88', embedding=None, metadata={'page_label': '1', 'file_name': 'MachineTranslationwithAttention.pdf', 'file_path': '/content/data/MachineTranslationwithAttention.pdf', 'file_type': 'application/pdf', 'file_size': 482135, 'creation_date': '2025-02-03', 'last_modified_date': '2025-02-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/355917108\nNeural Machine Translation with Attention\nTechnical Report · August 2021\nDOI: 10.13140/RG.2.2.29381.37607/1\nCITATIONS\n2\nREADS\n7,741\n2

In [7]:


# setup prompts - specific to StableLM
from llama_index.core import PromptTemplate

system_prompt = """<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [36]:
import torch

llm = HuggingFaceLLM(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

Settings.llm = llm
Settings.chunk_size = 1024

Supplied context_window 4096 is greater than the model's max input size 2048. Disable this warning by setting a lower context_window.


In [37]:


from llama_index.embeddings.huggingface import HuggingFaceEmbedding
Settings.embed_model =HuggingFaceEmbedding(model_name="BAAI/bge-large-zh-v1.5")

In [38]:
index=VectorStoreIndex.from_documents(documents)

In [39]:
query_engine=index.as_query_engine()
response=query_engine.query("what is attention?")

In [40]:
print(response)


The attention is a mechanism that allows a model to focus and pay more attention to relevant parts of the input sequence during the decoding phase, which can lead to better translation accuracy. When the encoder is not able to capture all the relevant information from the input sentence, the decoder can use the alignment between the encoder hidden state and dequeuing time at each steps of the decoding to predict the expected output. This method works for short sentences and long sentences.


In [41]:
response = query_engine.query("What did the author do growing up?")

In [42]:
print(response)


While I do not specifically remember my upbringing in a particular way, I can provide you with information about what growing up looks like based on the author's description:

Given: He spent his whole childhood in the woods

Answer: In his childhood, the author was most likely raised in a rural or mountainous area. This could indicate long periods of time spent outdoors, such as hiking or camping, which likely involved a variety of natural surroundings and environmental constraints. The childhood description could also reflect the author's experiences interacting with the natural world through these activities.


In [43]:

query_engine.query("how attention is different from rnn and lstm")

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Response(response='\nIn natural language translation (NMT), attention is a mechanism that facilitates the learning and memorization of past words in the input sentence, leading to better translation quality. It can provide context and highlight important information, helping to preserve the meaning of the input sentence while aligning with the target language. Compared to RNNs and LSTMs, attention in NMT is not only a way of encoding information but also an attention mechanism that can be used to process information at different stages of the encoder-decoder model, which leads to better results.', source_nodes=[NodeWithScore(node=TextNode(id_='299492b5-45a7-432a-aaef-7540eb8524cc', embedding=None, metadata={'page_label': '5', 'file_name': 'MachineTranslationwithAttention.pdf', 'file_path': '/content/data/MachineTranslationwithAttention.pdf', 'file_type': 'application/pdf', 'file_size': 482135, 'creation_date': '2025-02-03', 'last_modified_date': '2025-02-03'}, excluded_embed_metadata_k